In [1]:
from threading import Thread,Lock
import cv2
import time
from time import monotonic as timer
import os
import numpy as np
import queue
import threading
import random

from tensorflow.keras.models import load_model


from imutils.video import WebcamVideoStream
from imutils.video import FPS
import imutils
from IPython.display import display
from cv2 import *
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import matplotlib.image as mpimg

from object_detection.utils import ops as utils_ops
from object_detection.utils import visualization_utils as vis_util

In [2]:
from datetime import datetime

class CountsPerSec:
    """
    Class that tracks the number of occurrences ("counts") of an
    arbitrary event and returns the frequency in occurrences
    (counts) per second. The caller must increment the count.
    """

    def __init__(self):
        self._start_time = None
        self._num_occurrences = 0

    def start(self):
        self._start_time = datetime.now()
        return self

    def increment(self):
        self._num_occurrences += 1

    def countsPerSec(self):
        
        elapsed_time = (datetime.now() - self._start_time).total_seconds()
        return self._num_occurrences / elapsed_time

In [3]:
path = "/Users/guilhermeviveiros/Desktop/models/research/object_detection"
MODEL_NAME = 'hand_graph'
PATH_TO_CKPT = path + '/' + MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join(path+'/training', 'labelmap.pbtxt')
NUM_CLASSES = 1

In [4]:
detection_graph = tf.compat.v1.get_default_graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [5]:
from object_detection.utils import label_map_util
label_map_util.tf = tf.compat.v1
tf.gfile = tf.io.gfile

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [6]:
#Vou escolher random entre as frames que tenho num segundo até ter 12 frames
#Exemplo -> Se obtiver 36 frames por segundo retiro apenas 12 aleatoriamente
#        -> Este processo é repetido 3 vezes para os 3 segundos
#        -> Passo estes 3 gestos de 36 frames à rede e fico com a moda dos resultados

class Sub_Distributed_frames():
    import random
    
    def __init__(self,frames,thread_id):
        self.random1 = self.random(2,frames,thread_id)
        #self.random2 = self.random(10,frames,thread_id)
        #self.random3 = self.random(7,frames,thread_id)
        #self.l = len(frames)
        
    def getrandom(self):
        return self.random1
    
    #retorna 12 frames random entre as que o pc consegue tirar num segundo
    def random(self,seed,frames,thread_id):
        random.seed(seed)
        final_idx = []
        
        if(len(frames)) < 12:
            #print('Len de frames after: {}'.format(len(frames)))
            #print('Shape: {}'.format(frames[0].shape))
            return list(cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), (100,75))/255 for frame in frames)
        
        else:
            
            total_divisions = len(frames) // 12
        
            #vou de retirar 12 frames de todos os blocos de frames que o total_divisions me devolve
            for i in range(0,12):
            
                #lista currente de indices
                list_idx = [tmp for tmp in range(total_divisions*i,total_divisions*(i+1))]
                #escolho 1 indice aleatório
                idx = random.choices(list_idx, k=1)[0]
                #acrescento-o à lista
                final_idx.append(idx)
        
            return list(np.asarray(Image.fromarray(cv2.cvtColor(frames[i], cv2.COLOR_BGR2RGB)).resize((100,75)))/255. for i in final_idx )
    
        

In [8]:
class DistributedFrames():
    #frames ficam distribuidas em 3 listas, correspondetes a cada segundo
    #cada lista é utilizada por uma thread para processamento
    #utilizo listas, pois como demonstrado em cima são mais rápidas
    
    
    def __init__(self):
        
        self.handling_future = threading.Event()
        self.stopped = False
        
        self.list1 = [] #lista1
        self.list2 = [] #lista2
        self.list3 = [] #lista3
        
        self.current_list = 1
        
        self.e1 = threading.Event() #evento para a thread1
        self.e2 = threading.Event() #evento para a thread2
        self.e3 = threading.Event() #evento para a thread3
        
        self.t1 = Thread(target=self.thread_function, args=(1,self.e1), daemon=True) #thread1
        self.t2 = Thread(target=self.thread_function, args=(2,self.e2), daemon=True) #thread2 
        self.t3 = Thread(target=self.thread_function, args=(3,self.e3), daemon=True) #thread3
        
        self.t1.start()
        self.t2.start()
        self.t3.start()
        
    def stop(self):
        self.stopped = True
        
    def list_prox(self):
        
        cl = self.current_list
        
        global heartbeat_event
        global buffer
        #caso esteja na lista1, thread1 processa e siga
        if(cl == 1):
            
            self.e1.set()
            while(len(self.list1)!=12):
                #time.sleep(0.001)
                pass
            self.current_list +=1
            
            print('Heartbeat 1, thread1 done')
            heartbeat_event.set()
            
        #caso esteja na lista2, thread2 processa e siga
        elif(cl == 2):
        
            self.e2.set()
            while(len(self.list2)!=12):
                #time.sleep(0.001)
                pass
            self.current_list +=1
        
            print('Heartbeat 2, thread2 done')
            heartbeat_event.set()
            
                #caso esteja na lista3, thread3 processa, dá pull da thread mais antiga e volta à thread 1
        elif(cl == 3):
            #heartbeat_lock.acquire()
            self.e3.set()
            
            while(len(self.list3)!=12):
                #time.sleep(0.001)
                pass
        
            self.current_list +=1
            print('Heartbeat')
            heartbeat_event.set()
            #print('Ide vos todos foder')
            
        elif(cl == 4):
            #print('buffer length: {}'.format(len(buffer)))
            self.list1 = self.list2
            self.list2 = self.list3
            self.list3 = buffer
            buffer = []
            self.e3.set()
            #print('Heartbeat')
            heartbeat_event.set()
            
            
    def append(self,frame):
        global buffer
        l = self.current_list
        
        if(l == 1):
            self.list1.append(frame)
        elif(l == 2):
            self.list2.append(frame)
        elif(l == 3):
            self.list3.append(frame)
        elif(l == 4):
            buffer.append(frame)
        
    
    def set_event(self,id_thread):
        if(id_thread == 1):
            self.e1.set()
        elif(id_thread == 2):
            self.e2.set()
        elif(id_thread == 3):
            self.e3.set()
               
    #thread function
    #**
    #id_thread - representa o id_thread
    #e - representa o evento da thread
    #**
    def thread_function(self,id_thread,e):
        
        global processing_permission
        global processing_counter
        global frames
        global lock
        #enquanto não estiver pronto para executar espera    
        '''while not e.isSet():
            e.wait()'''
        if(id_thread == 1):
            
            while not self.e1.isSet():
                self.e1.wait()
            
            #thread 1 com as 12 frames
            self.list1 = Sub_Distributed_frames(self.list1,id_thread).random1
            print('after this, I sleep: thread {}'.format(id_thread))
            
            self.e1.clear()
            
            print("I'm done, thread 1 destroyed")
             
        elif(id_thread == 2):
                
            while not self.e2.isSet():
                self.e2.wait()
            
            #thread 2 com as 12 frames
            self.list2 = Sub_Distributed_frames(self.list2,id_thread).random1
            print('after this, I sleep: thread {}'.format(id_thread))
        
            self.e2.clear()
            
            print("I'm done, thread 2 destroyed")
            
            
        elif(id_thread == 3):
            
            #enquanto stop != True, processa
            while not self.stopped:
                while not self.e3.isSet():
                    self.e3.wait()
                
               
                #vai buscar as 12 frames
                self.list3 = Sub_Distributed_frames(self.list3,id_thread).random1
                
                
                
                if (processing_permission == True):
                    
                    processing_counter +=1
                    
                   
                if processing_counter == 2:
                    list_3 = self.list3.copy()
                    tam1 = len(self.list1)
                    tam2 = len(self.list2)
                    tam3 = len(self.list3)
                
                    print(tam1,tam2,tam3)
                    print(len(list_3),len(self.list2),len(self.list1))
                    
                    frames[0,(tam1+tam2):(tam1+tam2+tam3)] = np.asarray(list_3)
                    frames[0,tam1:(tam1+tam2)] = np.asarray(self.list2)
                    frames[0,0:tam1] = np.asarray(self.list1)
                    
                    main_model_event.set()
                    processing_counter = 0
                    processing_permission = False
                
                self.e3.clear()
                
            print("I'm done, thread 3 destroyed")

In [9]:
class TimeMarker():
    
    def __init__(self):
        self.stopped = False
        
    def start(self):
        Thread(target=self.set_marker, args=(),daemon=True).start()
        
    def set_marker(self):
        global lock
        global distributed_frames
        global heartbeat_event
        
        while not self.stopped:
            
            endtime = timer() + 1
            while timer() <= endtime:
                pass
            
            #depois de passar 1 segundo, vai buscar todas as frames de esse segundo, dá lock na class, retira só as 12 frames do segundo
            #e muda de lista, passa para a lista 2 correspondente ao 2 segundo.
            lock.acquire()
            
            distributed_frames.list_prox()   
            
            while not heartbeat_event.isSet():
                heartbeat_event.wait()
            heartbeat_event.clear()
            
            lock.release()
        
        print("Time Marker stopped, cya")
        
    def stop(self):
        self.stopped = True

In [10]:
class VideoGet():
    """
    Class that continuously gets frames from a VideoCapture object
    with a dedicated thread.
    """
    
    def __init__(self, src=0):
    
        self.stream = cv2.VideoCapture(src)
        (self.grabbed, self.frame) = self.stream.read()
        self.stopped = False
        
    def start(self):    
        Thread(target=self.get, args=(),daemon=True).start()
        return self

    def get(self):
        
        global lock
        global distributed_frames
        global frame
        global lock_main_thread
        global hand_thread_event
        
        start = datetime.now()
        
        #Se der stopped esta função termina e acaba a thread(função está em baixo, stop)
        while not self.stopped:
            #grabbed é no caso de erros inesperados, daí o self.stream.read em cima no init
            if not self.grabbed:
                self.stop()
            else:
                #apanha frames enquanto não tiver em stop
                (_ , self.frame) = self.stream.read()
        
                #lock 
                lock.acquire()
                
                self.frame = cv2.flip(self.frame,1)
                #print('Lock acquired')
                distributed_frames.append(self.frame)
                
                lock.release()
                
                frame = self.frame
                hand_thread_event.set()
                    
                
        print("Video Get stopped, cya :)")
        self.stream.release()
        cv2.destroyAllWindows()
        
            
    def stop(self):
        self.stopped = True

In [11]:
import matplotlib

def save_frames(frames):
    
    step = 0
    for frame in frames:
        
        if step < 10:
            name = str(step) + ".jpg"
            matplotlib.image.imsave("/Users/guilhermeviveiros/Desktop/LEI/Fotos/"+name, frame)
        else:
            name = str(step) + ".jpg"
            matplotlib.image.imsave("/Users/guilhermeviveiros/Desktop/LEI/Fotos/"+name, frame)
            
        step += 1

In [12]:
'''
When we pass this function to pycharm or .py use this code to terminate this function.
Doens't work in notebook, so for now i will stop after 30 iterations
try:
    while True:
        print("cousas")
        time.sleep(5)
except KeyboardInterrupt:
    pass
'''

def threadVideoGet(source=0):
    """
    Dedicated thread for grabbing video frames with VideoGet object.
    Main thread shows video frames.
    """
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()
    
    video_getter = VideoGet(source).start()
    print('Comecou o video getter')
    cps = CountsPerSec().start()
    time.sleep(1)
    start = datetime.now()
    global distributed_frames
    global frame
    global frames
    global processing_permission
    time_marker = TimeMarker().start()   
        
    i = 0
    
    with detection_graph.as_default():
    
        with tf.Session(graph=detection_graph) as sess:
                model = load_model('/Users/guilhermeviveiros/Desktop/simple_model/best_model_13_sgd_nesterov_800k_custom_dataset.h5')
                #model._make_predict_function() 
                
                print("Ready to start")
                
                while i < 200:         
                    i += 1
                    #fica à espera que a thread responsável pela captura de imagens, devolva uma frame
                    while not hand_thread_event.isSet():
                        hand_thread_event.wait()
                    
                    #print("Frame received")
                    
                    #Transformação da frame para ser compatível com o modelo de and detection                
                    image_np = imutils.resize(frame, width=600)
                    image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
                    
                
                    #Tudo do tensorflow
                    image_np_expanded = np.expand_dims(image_np,axis=0)
                    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                    scores = detection_graph.get_tensor_by_name('detection_scores:0')
                    classes = detection_graph.get_tensor_by_name('detection_classes:0')
                    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
                    
                    (boxes, scores, classes, num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})
                        
                    # Visualization of the results of a detection.
                    vis_util.visualize_boxes_and_labels_on_image_array(
                        image_np,
                        np.squeeze(boxes),
                        np.squeeze(classes).astype(np.int32),
                        np.squeeze(scores),
                        category_index,
                        use_normalized_coordinates=True,
                        max_boxes_to_draw = 1,
                        min_score_thresh=0.90,
                        line_thickness=8)
                    
                    #Display of frame
                    cv2.imshow("Frame", image_np)
                    
                    #Pronto para outra frame
                    hand_thread_event.clear()
                    
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                     
                    condition = scores[0][0]*100     
                        
                    print("Iteration " + str(i))
                    
                    if(condition >= 90):
                                            
                        print(condition)
                        
                        #Sinal para enviar os 3 segundos de frames entre a deteção da mão
                        processing_permission = True
                        
                        #Enquanto não estiverem processados o tempo que falta, espera
                        while not main_model_event.isSet():
                            main_model_event.wait()
                        
                        
                        print('Starting prediction')
                        
                        save_frames(frames[0])
                        l = model.predict(frames)
                        print(l)
                        l = np.argmax(l)
                        
                        if(l == 0): print("Doing other things")
                        elif(l==1): print("Swipping down")
                        elif(l==2): print("Swipping left")
                        elif(l==3): print("Zooming")
                        
                        #pronto para o próximo gesto
                        main_model_event.clear()
                        
                        #frames volta tudo a 0, sem frames
                        frames = np.zeros(shape=(1,36,75,100,3))
                        
                        #tempo para esvaziar buffers
                        time.sleep(2)
                        
                        print('Ready to detect again')
                        
                print("Leaving")
                #time_marker.stop()    
                video_getter.stop()
                cv2.destroyAllWindows()
                print("Done")
                #break
                        

In [13]:
#É o que organiza as 36 frames
distributed_frames = DistributedFrames()

# locks para trabalhar com as variaveis globais
lock = Lock()
lock_main_thread = Lock()

#variaveis global
frame = None
frames = np.zeros(shape=(1,36,75,100,3))
processing_counter = 0
debug_counter = 0
buffer = []


#Eventos para processamento de imagem e gesto
hand_thread_event = threading.Event() #informar sinal que encontrou a mão
main_model_event = threading.Event()
heartbeat_event = threading.Event() #Verficação das threads


processing_permission = False #mandar as 36 frames correspondentes aos 3 segundos

#Função principal que trata de tudo
threadVideoGet()

print("Leaving")
distributed_frames.stop()

print("All done")

Instructions for updating:
non-resource variables are not supported in the long term
Comecou o video getter
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
after this, I sleep: thread 1Heartbeat 1, thread1 done

I'm done, thread 1 destroyed
after this, I sleep: thread 2
I'm done, thread 2 destroyed
Heartbeat 2, thread2 done
Heartbeat
Ready to start
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
91.95849895477295
12 12 12
12 12 12
Starting prediction
[[7.3529975e-08 9.99996